In [1]:
import sys, re, os

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def charge_split(path):
    loader = PyPDFLoader(path)

    chunk_size = 700
    chunk_overlap = 150
    # initialize the splitter
    r_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    loader = loader.load_and_split(text_splitter=r_splitter)
    return loader

path = "4- TRAL-TRA-31520-E-009.pdf"
loader = charge_split(path)

In [2]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma

os.environ["OPENAI_API_TYPE"] = "api_key"
os.environ["OPENAI_API_KEY"] = "bb3d1d96f030475e918408f131285649"

vectordb = Chroma.from_documents(
  loader,
  embedding=AzureOpenAIEmbeddings(
      azure_endpoint='https://oai-llmca-poc-ecp-sbx.openai.azure.com/',
      azure_deployment='genia-embedding'),
  persist_directory='./data'
)
vectordb.persist()

In [8]:
help()


Welcome to Python 3.11's help utility!

If this is your first time using Python, you should definitely check out
the tutorial on the internet at https://docs.python.org/3.11/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To quit this help utility and
return to the interpreter, just type "quit".

To get a list of available modules, keywords, symbols, or topics, type
"modules", "keywords", "symbols", or "topics".  Each module also comes
with a one-line summary of what it does; to list the modules whose name
or summary contain a given string such as "spam", type "modules spam".




You are now leaving help and returning to the Python interpreter.
If you want to ask for help on a particular object directly from the
interpreter, you can type "help(object)".  Executing "help('string')"
has the same effect as typing a particular string at the help> prompt.


In [11]:
qa_chain.__dict__['memory'].load_memory_variables({})

{'name': None,
 'memory': AnswerConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='que elementos de competencia contiene la unidad de competencia 1? separe en bullets su respuesta'), AIMessage(content='\n- Inspeccionar condiciones operativas de la embarcación y equipos\n- Revisión del inventario de equipos y elementos de navegación\n- Alistar la embarcación de acuerdo con requerimientos del viaje, manual técnico y normativa fluvial\n- Describir los métodos y técnicas de alistamiento de la embarcación menor\n- Disponer de la embarcación fluvial menor para la navegación\n- Determinar distancias de ruta de acuerdo con destinos establecidos y técnicas de medición náutica')]), return_messages=True, llm=AzureOpenAI(client=<openai.resources.completions.Completions object at 0x000001A87DA8A990>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001A803084790>, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='

In [12]:
from langchain.chains import RetrievalQA
from langchain_openai import AzureOpenAI
from langchain.memory import ConversationTokenBufferMemory
from typing import Dict, Any

llm = AzureOpenAI(
        api_key='6286ae4863e54163bc669839b2531f1a',
        azure_endpoint='https://oai-llm-poc-ecp-sbx.openai.azure.com/',
        api_version='2023-12-01-preview',
        azure_deployment='Anie'
        )

class AnswerConversationBufferMemory(ConversationTokenBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerConversationBufferMemory, self).save_context(inputs,{'response': outputs['result']})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True,
    memory=AnswerConversationBufferMemory(llm=llm, return_messages=True, max_token_limit=500)
)
result = qa_chain.invoke({'query': 'que elementos de competencia contiene la unidad de competencia 1? separe en bullets su respuesta'})
print(result['result'])


- Inspeccionar condiciones operativas de la embarcación y equipos
- Revisión del inventario de equipos y elementos de navegación
- Acondicionamiento de motores
- Abastecimiento de combustible
- Reposición de fluidos
- Acondicionamiento de instrumentos y elementos de navegación
- Determinar distancias de ruta
- Confirmar los medios de salvamento a bordo


In [15]:
from langchain_core.output_parsers import StrOutputParser

help(StrOutputParser)

Help on class StrOutputParser in module langchain_core.output_parsers.string:

class StrOutputParser(langchain_core.output_parsers.transform.BaseTransformOutputParser)
 |  StrOutputParser(*, name: Optional[str] = None) -> None
 |  
 |  OutputParser that parses LLMResult into the top likely string.
 |  
 |  Method resolution order:
 |      StrOutputParser
 |      langchain_core.output_parsers.transform.BaseTransformOutputParser
 |      langchain_core.output_parsers.base.BaseOutputParser
 |      langchain_core.output_parsers.base.BaseLLMOutputParser
 |      langchain_core.runnables.base.RunnableSerializable
 |      langchain_core.load.serializable.Serializable
 |      pydantic.v1.main.BaseModel
 |      pydantic.v1.utils.Representation
 |      langchain_core.runnables.base.Runnable
 |      typing.Generic
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  parse(self, text: str) -> str
 |      Returns the input text with no changes.
 |  
 |  --------------------

In [14]:
qa_chain.invoke({'query': 'cual es mi nombre?'})

{'query': 'cual es mi nombre?',
 'history': [HumanMessage(content='que elementos de competencia contiene la unidad de competencia 1? separe en bullets su respuesta'),
  AIMessage(content='\n- Inspeccionar condiciones operativas de la embarcación y equipos\n- Revisión del inventario de equipos y elementos de navegación\n- Acondicionamiento de motores\n- Abastecimiento de combustible\n- Reposición de fluidos\n- Acondicionamiento de instrumentos y elementos de navegación\n- Determinar distancias de ruta\n- Confirmar los medios de salvamento a bordo'),
  HumanMessage(content='Hola gpt me llamo Gabriel'),
  AIMessage(content=' Hola Gabriel, ¿en qué puedo ayudarte hoy?'),
  HumanMessage(content='cual es mi nombre?'),
  AIMessage(content=' No se puede responder a esta pregunta ya que no hay suficiente contexto para saber su nombre.')],
 'result': ' No se puede responder a esta pregunta ya que no hay suficiente contexto para saber su nombre.',
 'source_documents': [Document(page_content='12 \n

In [12]:
vectordb = Chroma(
        'BID_POC',
        AzureOpenAIEmbeddings(
            azure_endpoint='https://oai-llmca-poc-ecp-sbx.openai.azure.com/',
            azure_deployment='genia-embedding'
        ),
        persist_directory='./data'
    )